In [1]:
import boa
import os
from dotenv import load_dotenv

In [2]:
load_dotenv()
boa.env.fork(f"https://eth-mainnet.g.alchemy.com/v2/{os.getenv('ALCHEMY_KEY')}")

# **Curve DAO Token**

CRV is a regular ERC20 token and contains the basic ERC20 methods. This notebook will only showcase some additionally added functionalities.

*Table of Content:*

1. [Querying Contract Informations](#1-querying-external-variables)
2. [Contract Ownership](#2-contract-admin)
3. [Minter](#3-minter)
4. [Change Name and Symbol](#4-changing-name-and-symbol)


In [3]:
crv_abi = boa.load_abi("../contracts/crv.json", name="Curve DAO Token")
crv = crv_abi.at("0xD533a949740bb3306d119CC777fa900bA034cd52")

## **1. Querying External Variables**

Querying public variables from the token contract is very easy:

In [4]:
admin = crv.admin()

print("Token Name:", crv.name())
print("Token Symbol:", crv.symbol())
print("Admin:", crv.admin())

Token Name: Curve DAO Token
Token Symbol: CRV
Admin: 0x40907540d8a6C65c637785e8f8B742ae6b0b9968


## **2. Contract Admin**

The Curve DAO Token contract has a `admin`. The functionality of the admin is limited, he can only change the name and symbol of the token or set a new admin. 
Current admin is the CurveOwnershipAgent ("0x40907540d8a6C65c637785e8f8B742ae6b0b9968"), which is controlled by the DAO (veCRV holders). 
Changing the `admin` of the contract therefore requires a successfully passed DAO vote.

- #### `set_admin(_admin: address):`

    Function to set/change the admin of the contract.

    Emits event: `SetAdmin`

    | Input      | Type   | Description |
    | ----------- | -------| ----|
    | `_admin` |  `address` | New Admin Address |


In [5]:
_admin = "0x7a16fF8270133F063aAb6C9977183D9e72835428"

print("Admin Before:", crv.admin())

with boa.env.prank("0x40907540d8a6C65c637785e8f8B742ae6b0b9968"):
    crv.set_admin(_admin)

print("Admin After:", crv.admin())

Admin Before: 0x40907540d8a6C65c637785e8f8B742ae6b0b9968
Admin After: 0x7a16fF8270133F063aAb6C9977183D9e72835428


## **3. Minter**

The Curve DAO Token has a `minter` variable. This address is the only address which is allowed to mint CRV tokens. The specific address is the GaugeController, which mints CRV tokens based on the current `rate` and distributes to the gauges according to the current gauge weights.

- #### `mint(_to: address, _value: uint256) -> bool:`

    Function to mint `_value (uint256)` and assign them to `_to (address)`.

    Returns: True (`bool`)

    Emits: `Transfer`

    | Input      | Type   | Description |
    | ----------- | -------| ----|
    | `_to` |  `address` | receiver of the minted tokens |
    | `_value` |  `uint256` | amount to mint  |

In [6]:
print("CRV Minter:", crv.minter())

CRV Minter: 0xd061D61a4d941c39E5453435B6345Dc261C2fcE0


In [7]:
_to = "0x7a16fF8270133F063aAb6C9977183D9e72835428"
_value = 10**18

try:    
    with boa.env.prank("0x7a16fF8270133F063aAb6C9977183D9e72835428"):
        crv.mint(_to, _value)
except:
    print("Transaction failed.")


Transaction failed.


If we sign the same transaction as the `minter`, the transaction passes. In production this is not possible. The minter is the GaugeController and this address can not be changed (see below).

In [8]:
to = "0x7a16fF8270133F063aAb6C9977183D9e72835428"
value = 10**18

before = crv.balanceOf(to)

try:    
    with boa.env.prank("0xd061D61a4d941c39E5453435B6345Dc261C2fcE0"):
        crv.mint(to, value)
except:
    print("Transaction failed.")

after = crv.balanceOf(to)

diff = after - before

print("CRV Token Balance Difference", diff / 1e18)


CRV Token Balance Difference 1.0


- #### `set_minter(_minter: address):`"

    Function to set the minter contract for the token.

    Emits: `SetMinter`

    | Input      | Type   | Description |
    | ----------- | -------| ----|
    | `_minter` |  `address` | New Minter Address |

Although there is a functuin to change the minter, is not actually possible to do so. The code checks the following two things:

- `assert msg.sender == self.admin  # dev: admin only`
- `assert self.minter == ZERO_ADDRESS  # dev: can set the minter only once, at creation`

The function must be called by the `admin` and the current minter address (`minter`) needs to be the `ZERO_ADDRESS` (0x0000000000000000000000000000000000000000).

In [9]:
try:
    with boa.env.prank("0x40907540d8a6C65c637785e8f8B742ae6b0b9968"):
        crv.set_minter("0x7a16fF8270133F063aAb6C9977183D9e72835428")
except:
    print("Transaction failed.")

Transaction failed.


The transaction fails because the `minter` address is not `ZERO_ADDRESS`:

In [10]:
print("CRV Minter Address:", crv.minter())

CRV Minter Address: 0xd061D61a4d941c39E5453435B6345Dc261C2fcE0


## **4. Changing Name and Symbol**

Name and symbol of the token can be changed ny the `admin` via the `set_name` function:

- #### `set_name(_name: String[64], _symbol: String[32])`

    Function to set a new name and symbol for the token.

    | Input     | Type         | Description |
    | --------- | ------------ | ----------- |
    | `_name`   | `String64]`  | New name    |
    | `_symbol` | `String[32]` | New Symbol  |

`Admin` of the Curve DAO Tokens is the CurveOwnershipAgent ("0x40907540d8a6C65c637785e8f8B742ae6b0b9968"). A change of name and/or symbol requires a successfully passed DAO vote. The transaction will revert if someone other than the admin tries to call the function.

In [11]:
_name = "Llama Token"
_symbol = "llama"
admin = crv.admin()

with boa.env.prank(admin):
    crv.set_name(_name, _symbol)

print("New Name:", crv.name())
print("New Symbol:", crv.symbol())

New Name: Llama Token
New Symbol: llama
